In [2]:
from __future__ import print_function

import base64
import mimetypes
import os
from email.message import EmailMessage

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials


SCOPES = ['https://mail.google.com/']

def send_with_attachment(To, From, Attachment_Path):
    """Create and insert a draft email with attachment.
       Print the returned draft's message and id.
      Returns: Draft object, including draft id and message meta data.

      Load pre-authorized user credentials from the environment.
      TODO(developer) - See https://developers.google.com/identity
      for guides on implementing OAuth2 for the application.
    """
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    try:
        # create gmail api client
        service = build('gmail', 'v1', credentials=creds)
        mime_message = EmailMessage()

        # headers
        mime_message['To'] = To
        mime_message['From'] = From
        mime_message['Subject'] = 'title'

        # text
        mime_message.set_content(
            'hello'
        )

        # attachment
        attachment_filename = Attachment_Path
        
        # guessing the MIME type
        type_subtype, _ = mimetypes.guess_type(attachment_filename)

        # Setting the MIME type manually
        type_subtype = 'application/pdf' # set it as pdf
        
        maintype, subtype = type_subtype.split('/')

        with open(attachment_filename, 'rb') as fp:
            attachment_data = fp.read()
        mime_message.add_attachment(attachment_data, maintype, subtype)

        encoded_message = base64.urlsafe_b64encode(mime_message.as_bytes()).decode()

        create_message = {
                'raw': encoded_message
        }
        # # pylint: disable=E1101
        # draft = service.users().drafts().create(userId="me",
        #                                         body=create_draft_request_body)\
        #     .execute()
        

        
        send_message = (service.users().messages().send
                        (userId="me", body=create_message).execute())
        
        print(F'Message Id: {send_message["id"]}')

    except HttpError as error:
        print(F'An error occurred: {error}')
        print('error :', To)
        draft = None
    return send_message

# Test

In [3]:
me = 'klee90@hawk.iit.edu'
test_file = '/Users/kunwoosmac/Desktop/Gmail_api/attachments/1.pdf'
send_with_attachment(me, me,test_file)

Message Id: 18ba8b103ebd19a5


{'id': '18ba8b103ebd19a5',
 'threadId': '18ba8b103ebd19a5',
 'labelIds': ['UNREAD', 'SENT', 'INBOX']}

# Load excel file for data

In [ ]:
from openpyxl import load_workbook
import numpy as np

source = ''
Wb = load_workbook(source)
ws = Wb['Sheet1']

certificate_number = np.array([ws.cell(x, 1).value for x in range(2, ws.max_row+1)])
attendee_name = np.array([ws.cell(x, 2).value for x in range(2, ws.max_row+1)])
email_adress = np.array([ws.cell(x, 3).value for x in range(2, ws.max_row+1)])

# Check
print('0~10')
certificate_number[0:10], attendee_name[0:10], email_adress[0:10]

# Send

In [ ]:
certificate_path = 'attachment file path'

for i in range(len(certificate_number)):
    print(str(attendee_name[i]), ' sent!')
    send_with_attachment(email_adress[i], me, certificate_path + '/' + str(certificate_number[i]) + '.pdf')